In [1]:
___Author___='LumberJack Jyss'

In [2]:
print('LumberJack BRUTAL AtidotCom BACKTEST\nLumberJack Jyss (c)')

LumberJack BRUTAL AtidotCom BACKTEST
LumberJack Jyss (c)


In [11]:
import backtrader as bt
import pandas as pd
import backtrader.feeds as btfeeds
import os
import datetime

In [10]:
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                                (order.executed.price,
                                 order.executed.value,
                                 order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))
                self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                         (trade.pnl, trade.pnlcomm))
    
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
    
        if self.order:
            return

        if not self.position:
            if self.dataclose[0] - self.dataclose[-1] > 500:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] - self.dataclose[-1] < -500:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
    
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    datapath = os.path.abspath(os.getcwd() + '/TBM_' + str(datetime.datetime.now().strftime("%Y_%m_%d")))
    
    # Create a Data Feed
    data = btfeeds.GenericCSVData(
        dataname=datapath,
        fromdate=datetime.datetime(2018, 1, 1),
        dtformat=('%Y-%m-%d %H:%M:%S'),
        datetime=0,
        high=2,
        low=3,
        open=1,
        close=4,
        volume=5,
        openinterest=-1,
        timeframe= bt.TimeFrame.Minutes,
        compression= 30
    )
    cerebro.adddata(data)
    cerebro.broker.setcash(1000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=0.05)
    cerebro.broker.setcommission(commission=0.005)
    print('Starting Balance: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Balance: %.2f' % cerebro.broker.getvalue())
    cerebro.plot()

Starting Balance: 1000.00


FileNotFoundError: [Errno 2] No such file or directory: '/Users/YTsBaCh/Documents/DATA SCIENCES/20190411 FOREX TREND/BTC_2019_04_15'